<img src="Tarjeta.png">

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Conectar-área-de-trabajo" data-toc-modified-id="Conectar-área-de-trabajo-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Conectar área de trabajo</a></span></li><li><span><a href="#Preparar-los-datos-para-el-experimento" data-toc-modified-id="Preparar-los-datos-para-el-experimento-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Preparar los datos para el experimento</a></span></li><li><span><a href="#Crear-un-script-de-entrenamiento" data-toc-modified-id="Crear-un-script-de-entrenamiento-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Crear un script de entrenamiento</a></span></li><li><span><a href="#Definición-del-environment" data-toc-modified-id="Definición-del-environment-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Definición del environment</a></span></li><li><span><a href="#Registrar-un-entorno" data-toc-modified-id="Registrar-un-entorno-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Registrar un entorno</a></span></li><li><span><a href="#Ver-los-entornos-registrados" data-toc-modified-id="Ver-los-entornos-registrados-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Ver los entornos registrados</a></span></li><li><span><a href="#Crear-un-cluster-de-proceso" data-toc-modified-id="Crear-un-cluster-de-proceso-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Crear un cluster de proceso</a></span></li><li><span><a href="#Ejecutar-experimento-en-cluster-remoto" data-toc-modified-id="Ejecutar-experimento-en-cluster-remoto-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Ejecutar experimento en cluster remoto</a></span></li></ul></div>

# Trabajar con entornos y procesos

Cuando ejecutas un script como un experimento en Azure ML, se necesita definir el contexto de ejecución para ejecutar el experimento. Esto esta formado por:
* El entorno de python que incluye todas las librerías y versiones.
* El Compute Target (Cluster de proceso) en el cual ejecutará. Este puede ser local o remoto.

## Conectar área de trabajo

In [1]:
import azureml.core
from azureml.core import Workspace

# Cargar el área de trabajo del fichero de configuración
ws = Workspace.from_config()
print('Versión de Azure ML {} y área de trabajo {}'.format(azureml.core.VERSION, ws.name))

Versión de Azure ML 1.32.0 y área de trabajo aml-nuclio


## Preparar los datos para el experimento

Se trabajará con el dataset de vinos cargado en la práctica anterior.

In [2]:
from azureml.core import Dataset

default_ds = ws.get_default_datastore()

if 'wine-dataset' not in ws.datasets:
    default_ds.upload_files(files=['./data/winequality.csv'], 
                        target_path='wine-data/', 
                        overwrite=True,
                        show_progress=True)

    # Crear un dataset tabular
    tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'wine-data/*.csv'))

    # Registrar el dataset tabular
    try:
        tab_data_set = tab_data_set.register(workspace=ws, 
                                name='wine-dataset',
                                description='datos del vino',
                                tags = {'format':'CSV'},
                                create_new_version=True)
        print('Dataset registrado.')
    except Exception as ex:
        print(ex)
else:
    print('Dataset ya existe.')

Dataset ya existe.


## Crear un script de entrenamiento

1. Carpeta para un nuevo experimento
2. Un script de entrenamiento que usa **scikit-learn** para el entrenamiento y **matplotlib** para visualizar la curva ROC.

In [3]:
import os

# Crear una carpeta para los entrenamiento
experiment_folder = 'vino_training_logistic'
os.makedirs(experiment_folder, exist_ok=True)
print(experiment_folder, 'Carpeta creada')

vino_training_logistic Carpeta creada


In [4]:
%%writefile $experiment_folder/diabetes_training.py
# Importar librerías
import argparse
from azureml.core import Run
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt

# Conseguir los argumentos del script
parser = argparse.ArgumentParser()
parser.add_argument('--regularization', type=float, dest='reg_rate', default=0.01, help='regularization rate')
parser.add_argument("--input-data", type=str, dest='training_dataset_id', help='training dataset')
args = parser.parse_args()

# Parametro de regularizacion
reg = args.reg_rate

# Contexto de la ejecución del experimento
run = Run.get_context()

# Cargar los datos
print("Cargando los datos...")
data = run.input_datasets['training_data'].to_pandas_dataframe()
# Cambiar formato de la variable
data["alcohol"] = pd.to_numeric(data["alcohol"], errors='coerce')
# Eliminar nulos
data = data.dropna()

# Separar features and target
features = ['fixed_acidity', 'volatile_acidity', 'citric_acid', 'residual_sugar', 'chlorides', 'free_sulfur_dioxide', 'total_sulfur_dioxide', 'density', 'ph', 'sulphates', 'alcohol']
X, y = data[features].values, data['top_quality'].values

# Separar train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Entrenamiento de una regresión logística
print('Entrenamiento de una Regresión Logística con ratio de regularización de', reg)
run.log('Ratio de regularizacion',  np.float(reg))
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# Calcular el accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# Calcular AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# Visualización curva ROC
fpr, tpr, thresholds = roc_curve(y_test, y_scores[:,1])
fig = plt.figure(figsize=(6, 4))
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
run.log_image(name = "ROC", plot = fig)
plt.show()

# Guardar el modelo entrenado en la carpeta outputs
os.makedirs('outputs', exist_ok=True)
joblib.dump(value=model, filename='outputs/vino_model.pkl')

run.complete()

Writing vino_training_logistic/diabetes_training.py


## Definición del environment

Cuando ejecutas un script de Python como un experimento en Azure ML, se crea un entorno de Conda. Azure ML tiene por defecto un entorno que incluye librerías comunes como **azureml-defaults** o **pandas** y **numpy**.

Sin embargo, también se puede definir tu propia especificación de Conda, añadiendo librerías usando **conda** o **pip**.

In [5]:
%%writefile $experiment_folder/experiment_env.yml
name: experiment_env
dependencies:
  # The python interpreter version.
  # Actualmente Azure ML solo soporta 3.5.2 y posteriores.
- python=3.6.2
- scikit-learn
- ipykernel
- matplotlib
- pandas
- pip
- pip:
  - azureml-defaults
  - pyarrow

Writing vino_training_logistic/experiment_env.yml


In [6]:
from azureml.core import Environment

# Creamos un entorno de python
experiment_env = Environment.from_conda_specification("experiment_env", experiment_folder + "/experiment_env.yml")

# Dejamos a Azure ML gestionar las dependencias
experiment_env.python.user_managed_dependencies = False 

# Visualizamos los detalles del entorno
print(experiment_env.name, 'definido.')
print(experiment_env.python.conda_dependencies.serialize_to_string())

experiment_env definido.
name: experiment_env
dependencies:
  # The python interpreter version.
  # Actualmente Azure ML solo soporta 3.5.2 y posteriores.
- python=3.6.2
- scikit-learn
- ipykernel
- matplotlib
- pandas
- pip
- pip:
  - azureml-defaults
  - pyarrow



In [8]:
from azureml.core import Experiment, ScriptRunConfig
from azureml.core.runconfig import DockerConfiguration
from azureml.widgets import RunDetails

# Conseguir el dataset de entrenamiento
diabetes_ds = ws.datasets.get("wine-dataset")

# Crear un script config
script_config = ScriptRunConfig(source_directory=experiment_folder,
                                script='diabetes_training.py',
                                arguments = ['--regularization', 0.1, # Parametro de regularizacion
                                             '--input-data', diabetes_ds.as_named_input('training_data')], # Referencia del dataset
                                environment=experiment_env,
                                docker_runtime_config=DockerConfiguration(use_docker=True)) # Usar docker para crear el entorno

# ejecutar experimento
experiment_name = 'nuclio-train-vino'
experiment = Experiment(workspace=ws, name=experiment_name)
run = experiment.submit(config=script_config)
RunDetails(run).show()
run.wait_for_completion()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

{'runId': 'nuclio-train-vino_1630335808_a7325eb1',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2021-08-30T15:07:22.091646Z',
 'endTimeUtc': '2021-08-30T15:07:39.792437Z',
 'properties': {'_azureml.ComputeTargetType': 'local',
  'ContentSnapshotId': 'bab826c5-1111-4eb2-abe5-c631a42c8b74'},
 'inputDatasets': [{'dataset': {'id': '044fff14-42f2-42bc-a9cd-6e12ceb11e9f'}, 'consumptionDetails': {'type': 'RunInput', 'inputName': 'training_data', 'mechanism': 'Direct'}}],
 'outputDatasets': [],
 'runDefinition': {'script': 'diabetes_training.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['--regularization',
   '0.1',
   '--input-data',
   'DatasetConsumptionConfig:training_data'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'local',
  'dataReferences': {},
  'data': {'training_data': {'dataLocation': {'dataset': {'id': '044fff14-42f2-42bc-a9cd-6e12ceb11e9f',
      'name': 'wine-dataset',
      'version':

In [9]:
# Métricas del log del experiemento
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

Ratio de regularizacion 0.1
Accuracy 0.8061643835616439
AUC 0.7705894776625426
ROC aml://artifactId/ExperimentRun/dcid.nuclio-train-vino_1630335808_a7325eb1/ROC_1630336048.png


ROC_1630336048.png
azureml-logs/60_control_log.txt
azureml-logs/70_driver_log.txt
logs/azureml/9_azureml.log
logs/azureml/dataprep/backgroundProcess.log
logs/azureml/dataprep/backgroundProcess_Telemetry.log
outputs/vino_model.pkl


## Registrar un entorno

In [11]:
# Register the environment
experiment_env.register(workspace=ws)

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20210615.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "experiment_env",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "dependencies": [
                "python=3.6.2",
                "scik

In [12]:
import os

# Crear una carpeta para un experimento
experiment_folder = 'vino_training_tree'
os.makedirs(experiment_folder, exist_ok=True)
print(experiment_folder, 'Carpeta creada')

vino_training_tree Carpeta creada


In [13]:
%%writefile $experiment_folder/vino_training.py
# Importar liberías
import argparse
from azureml.core import Run
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt

# Conseguir los argumentos del script 
parser = argparse.ArgumentParser()
parser.add_argument("--input-data", type=str, dest='training_dataset_id', help='training dataset')
args = parser.parse_args()

# Contexto de la ejecución del experimento
run = Run.get_context()

# Cargar los datos
print("Cargando los datos...")
data = run.input_datasets['training_data'].to_pandas_dataframe()
# Cambiar formato de la variable
data["alcohol"] = pd.to_numeric(data["alcohol"], errors='coerce')
# Eliminar nulos
data = data.dropna()

# Separar features and target
features = ['fixed_acidity', 'volatile_acidity', 'citric_acid', 'residual_sugar', 'chlorides', 'free_sulfur_dioxide', 'total_sulfur_dioxide', 'density', 'ph', 'sulphates', 'alcohol']
X, y = data[features].values, data['top_quality'].values

# Separar train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Entrenamiento de un arbol de decision
print('Entrenando un decision tree model')
model = DecisionTreeClassifier().fit(X_train, y_train)

# Calcular el accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# Calcular AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# Visualización curva ROC
fpr, tpr, thresholds = roc_curve(y_test, y_scores[:,1])
fig = plt.figure(figsize=(6, 4))
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
run.log_image(name = "ROC", plot = fig)
plt.show()

# Guardar el modelo entrenado en la carpeta outputs
os.makedirs('outputs', exist_ok=True)
joblib.dump(value=model, filename='outputs/vino_model.pkl')

run.complete()

Writing vino_training_tree/vino_training.py


Como se ha registrado el entorno, se puede usar para ejecutar nuevos experimentos.

In [14]:
# Consigue el entorno registrado
registered_env = Environment.get(ws, 'experiment_env')

# Conseguir el dataset de entrenamiento
diabetes_ds = ws.datasets.get("wine-dataset")

# Crear un script config
script_config = ScriptRunConfig(source_directory=experiment_folder,
                              script='vino_training.py',
                              arguments = ['--input-data', diabetes_ds.as_named_input('training_data')], # Referencia del dataset
                              environment=registered_env,
                              docker_runtime_config=DockerConfiguration(use_docker=True)) 

# ejecutar experimento
experiment_name = 'nuclio-train-vino'
experiment = Experiment(workspace=ws, name=experiment_name)
run = experiment.submit(config=script_config)
RunDetails(run).show()
run.wait_for_completion()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

{'runId': 'nuclio-train-vino_1630336403_f006160c',
 'target': 'local',
 'status': 'Finalizing',
 'startTimeUtc': '2021-08-30T15:13:25.136086Z',
 'properties': {'_azureml.ComputeTargetType': 'local',
  'ContentSnapshotId': '7a80db1e-936e-4205-9a58-abff7f35df95'},
 'inputDatasets': [{'dataset': {'id': '044fff14-42f2-42bc-a9cd-6e12ceb11e9f'}, 'consumptionDetails': {'type': 'RunInput', 'inputName': 'training_data', 'mechanism': 'Direct'}}],
 'outputDatasets': [],
 'runDefinition': {'script': 'vino_training.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['--input-data', 'DatasetConsumptionConfig:training_data'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'local',
  'dataReferences': {},
  'data': {'training_data': {'dataLocation': {'dataset': {'id': '044fff14-42f2-42bc-a9cd-6e12ceb11e9f',
      'name': 'wine-dataset',
      'version': '1'},
     'dataPath': None,
     'uri': None},
    'mechanism': 'Direct',
    'envi

This time the experiment runs more quickly because a matching environment has been cached from the previous run, so it doesn't need to be recreated on the local compute. However, even on a different compute target, the same environment would be created and used - ensuring consistency for your experiment script execution context.

Let's look at the metrics and outputs from the experiment.

In [15]:
# Métricas del log del experimento
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

Accuracy 0.8226027397260274
AUC 0.766768377407988
ROC aml://artifactId/ExperimentRun/dcid.nuclio-train-vino_1630336403_f006160c/ROC_1630336411.png


ROC_1630336411.png
azureml-logs/60_control_log.txt
azureml-logs/70_driver_log.txt
logs/azureml/10_azureml.log
logs/azureml/dataprep/backgroundProcess.log
logs/azureml/dataprep/backgroundProcess_Telemetry.log
outputs/vino_model.pkl


## Ver los entornos registrados


In [18]:
from azureml.core import Environment

envs = Environment.list(workspace=ws)
for env in envs:
    print("Nombre",env)

Nombre experiment_env
Nombre AzureML-VowpalWabbit-8.8.0
Nombre AzureML-PyTorch-1.3-CPU
Nombre AzureML-Minimal
Nombre AzureML-Tutorial
Nombre AzureML-Dask-CPU
Nombre AzureML-Dask-GPU
Nombre AzureML-Triton
Nombre AzureML-PyTorch-1.6-GPU
Nombre AzureML-PyTorch-1.6-CPU
Nombre AzureML-TensorFlow-2.3-CPU
Nombre AzureML-DeepSpeed-0.3-GPU
Nombre AzureML-TensorFlow-2.3-GPU
Nombre AzureML-tritonserver-21.02-py38-inference
Nombre AzureML-sklearn-0.24-ubuntu18.04-py37-cpu
Nombre AzureML-pytorch-1.8-ubuntu18.04-py37-cuda11-gpu
Nombre AzureML-pytorch-1.7-ubuntu18.04-py37-cuda11-gpu
Nombre AzureML-pytorch-1.9-ubuntu18.04-py37-cuda11-gpu
Nombre AzureML-tensorflow-2.4-ubuntu18.04-py37-cuda11-gpu
Nombre AzureML-lightgbm-3.2-ubuntu18.04-py37-cpu
Nombre AzureML-xgboost-0.9-ubuntu18.04-py37-cpu-inference
Nombre AzureML-onnxruntime-1.6-ubuntu18.04-py37-cpu-inference
Nombre AzureML-pytorch-1.6-ubuntu18.04-py37-cpu-inference
Nombre AzureML-mlflow-ubuntu18.04-py37-cpu-inference
Nombre AzureML-tensorflow-2.4-ub

In [19]:
for env in envs:
    if env.startswith("AzureML"):
        print("Nombre",env)
        print("librerías", envs[env].python.conda_dependencies.serialize_to_string())

Nombre AzureML-VowpalWabbit-8.8.0
librerías channels:
- conda-forge
dependencies:
- python=3.6.2
- pip:
  - azureml-core==1.19.0
  - azureml-defaults==1.19.0
  - azureml-dataset-runtime[fuse,pandas]
name: azureml_769be4b756b756954fa484d1287d5153

Nombre AzureML-PyTorch-1.3-CPU
librerías channels:
- conda-forge
dependencies:
- python=3.6.2
- pip=20.2.4
- pip:
  - azureml-core==1.21.0.post1
  - azureml-defaults==1.21.0
  - azureml-telemetry==1.21.0
  - azureml-train-restclients-hyperdrive==1.21.0
  - azureml-train-core==1.21.0
  - torch==1.3
  - torchvision==0.4.1
  - mkl==2018.0.3
  - horovod==0.18.1
  - tensorboard==1.14.0
  - future==0.17.1
name: azureml_a02f4fa469cd8066bd6e2f219433318d

Nombre AzureML-Minimal
librerías channels:
- conda-forge
dependencies:
- python=3.6.2
- pip:
  - azureml-core==1.33.0.post1
  - azureml-defaults==1.33.0
name: azureml_4b0b8e84d3cc562fa0a88e2d4ed4bc93

Nombre AzureML-Tutorial
librerías channels:
- anaconda
- conda-forge
dependencies:
- python=3.6.2
- p

AttributeError: 'NoneType' object has no attribute 'serialize_to_string'

## Crear un cluster de proceso

En muchas ocasiones, los recursos computacionales locales no son suficientes para tu proceso o tardan demasiado en ejecutar. Con AzureML se pueden crear recursos para ejecutar procesos en la nube.

In [27]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "NuclioCluster"

try:
    # Validar si existe
    training_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Cluster de proceso encontrado,')
except ComputeTargetException:
    # Si no existe, se crea
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        training_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        training_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)

InProgress......
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Ejecutar experimento en cluster remoto

Volvemos a ejecutar el experimento anterior, pero esta vez en el cluster de proceso creado.

> **Nota**: el experimento tardarña más porque el contenedor de la imagen debe de construirse con el entorno de conda. Para este experimento es inficiente, pero para experimentos con muchos más datos hace mucho más escalable el trabajo un data scientist.

In [28]:
# Crear un script config
script_config = ScriptRunConfig(source_directory=experiment_folder,
                                script='vino_training.py',
                                arguments = ['--input-data', diabetes_ds.as_named_input('training_data')],
                                environment=registered_env,
                                compute_target=cluster_name) 

# ejecutar experimento
experiment_name = 'nuclio-train-vino'
experiment = Experiment(workspace=ws, name=experiment_name)
run = experiment.submit(config=script_config)
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

Se puede ver el estado de cluster de proceso con el código de abajo.

In [29]:
cluster_state = training_cluster.get_status()
print(cluster_state.allocation_state, cluster_state.current_node_count)

Steady 0


El estado puede tardar en cambiar de *steady* a *resizing*.

In [30]:
run.wait_for_completion()

{'runId': 'nuclio-train-vino_1630337250_6d96f979',
 'target': 'NuclioCluster2',
 'status': 'Finalizing',
 'startTimeUtc': '2021-08-30T15:42:01.857211Z',
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': '7a80db1e-936e-4205-9a58-abff7f35df95',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [{'dataset': {'id': '044fff14-42f2-42bc-a9cd-6e12ceb11e9f'}, 'consumptionDetails': {'type': 'RunInput', 'inputName': 'training_data', 'mechanism': 'Direct'}}],
 'outputDatasets': [],
 'runDefinition': {'script': 'vino_training.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['--input-data', 'DatasetConsumptionConfig:training_data'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'NuclioCluster2',
  'dataReferences': {},
  'data': {'training_data': {'dataLocation': {'dataset': {'id': '044fff14-42f2-42bc-a9cd-6e12ceb11

In [31]:
# Métricas del log del experimento
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

Accuracy 0.8273972602739726
AUC 0.7571058569824701
ROC aml://artifactId/ExperimentRun/dcid.nuclio-train-vino_1630337250_6d96f979/ROC_1630338178.png


ROC_1630338178.png
azureml-logs/20_image_build_log.txt
azureml-logs/55_azureml-execution-tvmps_d8fa2802b74f936302e4fabf17ccc27f551b145c9372e0c9f41c68f30cb6ec96_d.txt
azureml-logs/65_job_prep-tvmps_d8fa2802b74f936302e4fabf17ccc27f551b145c9372e0c9f41c68f30cb6ec96_d.txt
azureml-logs/70_driver_log.txt
azureml-logs/75_job_post-tvmps_d8fa2802b74f936302e4fabf17ccc27f551b145c9372e0c9f41c68f30cb6ec96_d.txt
azureml-logs/process_info.json
azureml-logs/process_status.json
logs/azureml/94_azureml.log
logs/azureml/dataprep/backgroundProcess.log
logs/azureml/dataprep/backgroundProcess_Telemetry.log
logs/azureml/job_prep_azureml.log
logs/azureml/job_release_azureml.log
outputs/vino_model.pkl


In [33]:
from azureml.core import Model

# Registrar el modelo
run.register_model(model_path='outputs/vino_model.pkl', model_name='vino_model',
                   tags={'Training context':'Compute cluster'}, properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

# List registered models
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

vino_model version: 4
	 Training context : Compute cluster
	 AUC : 0.7571058569824701
	 Accuracy : 0.8273972602739726


diabetes_model version: 3
	 Training context : Compute cluster
	 AUC : 0.7571058569824701
	 Accuracy : 0.8273972602739726


vino_model version: 3
	 Training context : Tabular dataset
	 AUC : 0.7700142283878569
	 Accuracy : 0.8068493150684931


vino_model version: 2
	 Training context : Parameterized script
	 AUC : 0.7698252576116318
	 Accuracy : 0.8068493150684931


vino_model version: 1
	 Training context : Script
	 AUC : 0.7697446671335356
	 Accuracy : 0.8047945205479452


diabetes_model version: 2
	 Training context : Parameterized script
	 AUC : 0.8483198169063138
	 Accuracy : 0.774


diabetes_model version: 1
	 Training context : Script
	 AUC : 0.8484929598487486
	 Accuracy : 0.774


AutoML2c70a24fc0 version: 1


